## Components analysis + clustering
* Dimension reduction (PCA)
* Understanding data structure (clustering)
* Decomposing sources of variance (ICA)

In [ ]:
# load some data
import os
import pandas
import numpy as np

wdir = os.path.join(os.getcwd(),'stuff')
if not os.path.isdir(wdir) and os.path.isfile(os.path.join(os.getcwd(),'stuff.tar.gz')):
    os.system('tar -xvzf %s'%os.path.join(os.getcwd(),'stuff.tar.gz'))

wdir = os.path.join(os.getcwd(),'stuff')
df = pandas.read_csv(os.path.join(wdir,'MAIN_hippocampus_sample_info.csv'))
jnk = np.load(os.path.join(wdir,'gxp.npz'))
gxp = pandas.DataFrame(jnk['arr_0'].T)
gxp.drop(gxp.index[0],inplace=True)

This is Allen Human Brain Atlas data. Specifically, it is a set of 170 brain samples inside the hippocampus (from 6 different donors). For each sample, we have microarray data summarizing the normalized expression level of 60k probes, each probe corresponding to a gene

In [ ]:
gxp.shape

In [ ]:
#gxp.head()

In [ ]:
# Let's reduce the data with PCA
from sklearn.decomposition import PCA

pca = PCA(n_components = 170, random_state=123)
mod = pca.fit(gxp)

In [ ]:
# how much variance can we explain per component
import matplotlib.pyplot as plt
plt.close()
plt.plot(mod.explained_variance_ratio_)
plt.xlabel('Number of components')
plt.ylabel('Percentage of total variance explained')
plt.show()

In [ ]:
# lets zoom in a bit
plt.close()
plt.plot(mod.explained_variance_ratio_[:25])
plt.xlabel('Number of components')
plt.ylabel('Percentage of total variance explained')
plt.show()

It looks like we can explain quite a bit of the total variance of the data using a fraction of the components...

In [ ]:
# How much variance can we explain with just (arbitrarily) 25 components?
pca25 = PCA(n_components = 25, random_state=123)
mod = pca25.fit(gxp)
sum(mod.explained_variance_ratio_)

In [ ]:
# Now lets transform our data
gxp_PCA = mod.transform(gxp)
gxp_PCA.shape

Now, instead of having 60k features, we have 25 features, but we're explaining 3/4 of the total variance of the 60k

In [ ]:
# the component matrix is available to understand the importance of features to the components
mod.components_.shape

# and you can always back-transform the data
# mod.inverse_transform

#### Use clustering to explore the natural structure of our data 

In [ ]:
import seaborn as sns
# Using a simple clustering plot, we can look at how samples group 
# based on covaryiance of features
plt.close()
sns.clustermap(gxp_PCA, col_cluster=False, row_cluster=True)
plt.show()

the sklearn.cluster module has a number of clustering algorithms that are accessible and simple to use



In [ ]:
from sklearn import cluster

In [ ]:
cluster.

In [ ]:
# as an example, I'll run quick k-means clustering on the same data as above
# note that I'm setting the random state -- important because this a stochastic process
kmean = cluster.KMeans(n_clusters=4, random_state=123)
kmeans_gxp = kmean.fit(gxp_PCA)

In [ ]:
# we can easy retrieve the resulting cluster labels for each input sample
kmeans_gxp.labels_

In [ ]:
plt.close()
pandas.crosstab(pandas.Series(kmeans_gxp.labels_),
               df.structure_acronym
               ).plot.bar()
plt.show()

In [ ]:
# Here, we'll use a fancy clustering algorithm to reduce the date further in order
# to visualize the natural clustering of the data in three dimensions
import umap
nbr = 5
dist = 0.01
embedding = umap.UMAP(n_components=3, n_neighbors=nbr, min_dist=dist, random_state=123
                     ).fit_transform(gxp_PCA)

In [ ]:
# to see if the hippocampal samples cluster based on the subfield they belong to, we'll label
# each sample by which subfield it belongs to
code = dict(zip(df.structure_acronym.unique(),
               range(len(df.structure_acronym.unique()))))
labs = [code[x] for x in df.structure_acronym.tolist()]
lmap = dict(zip(code.values(),code.keys()))

In [ ]:
# and now we'll plot the relationship
import numpy as np
import plotly

traces = list()
for lab in np.unique(labs):
    l_index = [x for x in range(len(labs)) if labs[x] == lab]
    l_embed = embedding[l_index]
#     color_str = str(int('0x' + color_dict[lab][1:3].upper(),16)) + ',' + \
#                    str(int('0x' + color_dict[lab][3:5].upper(),16)) + ',' + \
#                    str(int('0x' + color_dict[lab][5:7].upper(),16))
    #color_str = plt.cm.tab10.colors[lab]
    color_str = 'rgba %s'%str(tuple([x - 0.0000001 for x in plt.cm.tab10(lab) if x > 0]))
    temp_trace = plotly.graph_objs.Scatter3d(x=l_embed[:,0],
                                            y = l_embed[:,1],
                                            z = l_embed[:,2],
                                            name = lmap[lab],
                                            mode = 'markers',
                                            marker = dict(size=5,
                                                          color=color_str
                                                         )
                                            )
    traces.append(temp_trace)

layout = plotly.graph_objs.Layout(margin = dict(l=0,r=0,b=0,t=0))

fig = plotly.graph_objs.Figure(data=traces, layout=layout)
plotly.offline.plot(fig, filename='clusters.html')
    

In [ ]:
# here is a non-interactive version of the same thing
from mpl_toolkits.mplot3d import Axes3D

plt.close()
fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)
ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], c = labs, s=20)
plt.show()
#plt.savefig(os.path.join(outdir,'umap_%s_%s.pdf'%(nbr,dist)))

#### Documentation

sklearn clustering documentaton: http://scikit-learn.org/stable/modules/clustering.html

sklearn signal decomposition documentation: http://scikit-learn.org/stable/modules/decomposition.html

installing sklearn: http://scikit-learn.org/stable/install.html

umap: https://github.com/lmcinnes/umap

If you have any further questions or think I can be of help, my brainhack slack handle is jvogel